In [ ]:
import pandas as pd

In [ ]:
games = pd.read_csv("nba_games.csv", index_col=0)

In [ ]:
games.head()

In [ ]:
games.shape

In [ ]:
games["team"].value_counts()

In [ ]:
games[games["team"] == "GSW"].sort_values("date")

Counts how many games are in each season 

In [ ]:
games["season"].value_counts()

In [ ]:
games.dtypes

Deleting column that is not needed 

In [ ]:
del games["mp.1"]

Converting existing column to a date time then overiding the existing column with the date time.

In [ ]:
games["date"] = pd.to_datetime(games["date"], format="%d/%m/%Y")

In [ ]:
games

In [ ]:
games.dtypes

Creating predictors for machine learning 
home says wheteher the team played at a home game or an away game

concatenates the home category in the table 

In [ ]:
games ["venue_code"] = games["home"].astype("category").cat.codes

This creates the code for each opponent

In [ ]:
games ["opp_code"] = games["team_opp"].astype("category").cat.codes

In [ ]:
games ["day_code"] = games ["date"].dt.dayofweek

In [ ]:
games

The following code sets up a target to predict

In [ ]:
games["target"] = (games["won"] == True).astype("int")

In [ ]:
games

# Training our neural network model

Scaling data having vector X as [0], or standardise to have mean 0 and variance 1(ask about this)
StandardScaler is used for standardisation

In [ ]:
from sklearn.preprocessing import StandardScaler

Multi-layer Perceptron (MLP) is a supervised learning algorithm that learns a function 
 by training on a dataset, where 
 is the number of dimensions for input and 
 is the number of dimensions for output. Given a set of features 
 and a target 
, it can learn a non-linear function approximator for either classification or regression. It is different from logistic regression, in that between the input and the output layer, there can be one or more non-linear layers, called hidden layers. Figure 1 shows a one hidden layer MLP with scalar output.
Reference: https://scikit-learn.org/stable/modules/neural_networks_supervised.html#tips-on-practical-use

This Class MLPClassifier implements a multi-layer perceptron (MLP) algorithm that trains using backpropagation(taking the error rate of a forward propagation and feeding this loss backwards through the neural network layer to fine-tune the weights, key to neural net training)
Reference: https://builtin.com/machine-learning/backpropagation-neural-network

In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

MLP trains on two arrays: array X of size (n_samples, n_features), which holds the training samples represented as floating point feature vectors; and array y of size (n_samples,), which holds the target values (class labels) for the training samples:

In [ ]:
clf = MLPClassifier(random_state=1, max_iter=300)

In [ ]:
predictors = ["venue_code","opp_code","day_code"]
X_train = games[games["date"]<'2020-10-11']
y_train = games[games["date"]<'2020-10-11']
X_test = games[games["date"]>'2020-10-11']
y_test = games[games["date"]>'2020-10-11']

In [ ]:
clf.fit(X_train[predictors], y_train["target"])

In [ ]:
preds = clf.predict(X_test[predictors])

# Performance metrics for our neural network model

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
acc = accuracy_score(X_test["target"], preds)

In [ ]:
acc

In [ ]:
combined = pd.DataFrame(dict(actual=X_test["target"], prediction=preds))

In [ ]:
pd.crosstab(index= combined["actual"],columns=combined["prediction"])

470 times the model was correct, 320 times the model was wrong 
401 times the model was correct, 389 times the model was wrong

In [ ]:
from sklearn.metrics import accuracy_score
acc = accuracy_score(X_test["target"], preds)

In [ ]:
from sklearn.metrics import precision_score
precision_score(X_test["target"], preds)

In [ ]:
from sklearn.metrics import recall_score
recall_score(X_test["target"], preds)

In [ ]:
from sklearn.metrics import f1_score
f1_score(X_test["target"], preds)

In [ ]:
from sklearn.metrics import confusion_matri
confusion_matrix(X_test["target"], preds)

# Improvement of our neural network model with rolling averages

Improving the precison with rolling averages- by splitting the data set up in teams
This groups the data set into team statistics 

In [ ]:
grouped_games = games.groupby("team")

In [ ]:
group = grouped_games.get_group("GSW")

In [ ]:
group

The games of GSW (Golden State Warriors)if the prediction is on the fourth game the model should look at the previous three games to help make better predictions. For example if the team is on a losing streak or a winning streak

In [ ]:
def rolling_averages(group, cols, new_cols):
    group = group.sort_values("date")
    #closed = ''left' only have the rolling averages of the previous weeks not the week we are trying to calculate for
    rolling_stats = group[cols].rolling(3,closed='left').mean()
    group[new_cols] = rolling_stats
    #removes all of the rows that have missing values 
    group = group.dropna(subset= new_cols)
    return group

Using the same roling averages as the ones used for Random Forest

In [ ]:
cols = ["fg","3p","ft","ast","stl","blk","tov", "pts"]
new_cols = [f"{c}_rolling" for c in cols]

In [ ]:
new_cols

These new columns are just for the LAL games 

In [ ]:
rolling_averages(group, cols, new_cols)

After getting the original nba_games data Frame and grouped by teams which creates one data frame for each team in the data. Then the lambda function was applied to compute the rolling averages.

In [ ]:
games_rolling = games.groupby("team").apply(lambda x:rolling_averages(x, cols, new_cols)) 

Rolling averages is applied to the entire data set

In [ ]:
games_rolling

Dropping the team name as a seperate level in the pandas index. for a multi level index you would need to use two values to access each row which was not needed

In [ ]:
games_rolling = games_rolling.droplevel('team')

In [ ]:
games_rolling 

In [ ]:
games_rolling.index = range(games_rolling.shape[0])

In [ ]:
games_rolling

In [ ]:
def make_predictions(data,predictors):
    X_train = data[data["date"]<'2022-01-01']
    y_test = data[data["date"]>'2022-01-01']
    clf.fit(X_train[predictors], X_train["target"])
    preds = clf.predict(y_test[predictors])
    combined = pd.DataFrame(dict(actual=y_test["target"], predicted=preds), index=y_test.index)
    precision = precision_score(y_test["target"], preds)
    return combined, precision 

In [ ]:
combined, precision = make_predictions(games_rolling, predictors + new_cols)

In [ ]:
precision

In [ ]:
combined

In [ ]:
#merges rows based on index
combined = combined.merge(games_rolling[["date", "team", "team_opp", "won"]], left_index=True, right_index=True)

In [ ]:
#information about the team that played their opponent and the result
combined

Combining home and away predictions

In [ ]:
#to make sure the names of the home and away team are the same 
#the Dataset already has the same names for home and away teams
#for future reference if names need to be normalised
class MissingDict(dict):
    __missing__ = lambda self, key: key
    
map_values = {
    
    "Golden State Warriors": "GSW",
    "Lakers": "LAL",
    "Boston Celtics": "BOS"
    
}
mapping = MissingDict(**map_values)

In [ ]:
mapping["Golden State Warriors"]

In [ ]:
combined["new_team"] = combined["team"].map(mapping)

In [ ]:
combined

This makes sure that the predictions are the same on both sides 

In [ ]:
merged = combined.merge(combined, left_on=["date","new_team"], right_on=["date","team_opp"])

In [ ]:
merged

In [ ]:
#def show_missing(games):
 #   """Return a Pandas dataframe describing the contents of a source dataframe including missing values."""
    
  #  variables = []
   # dtypes = []
    #count = []
#    unique = []
#    missing = []
 #   pc_missing = []
    
  #  for item in games.columns:
   #     variables.append(item)
    #    dtypes.append(games[item].dtype)
     #   count.append(len(games[item]))
      #  unique.append(len(games[item].unique()))
       # missing.append(games[item].isna().sum())
        #pc_missing.append(round((games[item].isna().sum() / len(games[item])) * 100, 2))

#    output = pd.DataFrame({
 #       'variable': variables, 
  #      'dtype': dtypes,
   #     'count': count,
    #    'unique': unique,
     #   'missing': missing, 
      #  'pc_missing': pc_missing
    #})    
        
    #return output

In [ ]:
#show_missing(games)

In [ ]:
nulls = pd.isnull(games)
nulls = nulls.sum()
nulls = nulls[nulls > 0]
nulls

In [ ]:
merged

In [ ]:
merged[(merged["prediction_x"] == 1) & (merged["prediction_y"] == 0)]["actual_x"].value_counts()

The accuracy is 76.5% from merging both sides of the match together however manyrows were unable to be recovered to get an accuracy on all of the data frames rows.
Using more seasons could improve accuracy
Using more columns in the data frame can improve accuracy 
Look at both current team and opponent team records for the season

In [ ]:
combined, accuracy = make_predictions(games_rolling, predictors + new_cols)

In [ ]:
def make_predictions(data,predictors):
    X_train = data[data["date"]<'2022-01-01']
    y_test = data[data["date"]>'2022-01-01']
    clf.fit(X_train[predictors], X_train["target"])
    preds = clf.predict(y_test[predictors])
    combined = pd.DataFrame(dict(actual=y_test["target"], predicted=preds), index=y_test.index)
    accuracy = accuracy_score(y_test["target"], preds)
    return combined, accuracy  

In [ ]:
merged, accuracy = make_predictions(games_rolling, predictors + new_cols)

In [ ]:
accuracy

#### 55% accuracy with rolling averages